正式论文实验：构造并使用router

In [40]:
import json
datas=[]
with open('MATH_TEST.jsonl','r',encoding='utf-8') as f:
    lines=f.readlines()
    for line in lines:
        data=json.loads(line)
        # if data['split']=='test':
        datas.append(data)
datas2=[]
with open('final_data/MATH_TEST_LATEX_0.jsonl','r',encoding='utf-8') as f:
    lines=f.readlines()
    for line in lines:
        data=json.loads(line)
        # if data['split']=='test':
        datas2.append(data)

In [42]:
import re
# 根据编号进行分组
# 正则表达式匹配编号和随后的文本
def seg_text(text):
    pattern = r'\[\d+\].*?(?=\n\[|\Z)'
    matches = re.finditer(pattern, text, re.DOTALL)
    match_list = [m.group() for m in matches]
    return match_list


In [41]:
# 提取latex提示词
latex=[]
for i in range(len(datas2)):
    latex.append(datas2[i]['latex'])

In [43]:
import random
seg_text(datas2[random.randint(0,len(latex)-1)]['latex'])

['[1] Define the set comprehension condition precisely',
 '[2] Understand the floor division constraints',
 '[3] Count the number of elements in the set that satisfy the condition',
 '[4] Verify that the constraint creates unique divisions for 4, 5, 6',
 '[5] Determine the conditions for uniqueness',
 '[6] Check if the set count matches the expected 80',
 '[7] Develop a systematic method to count these unique numbers',
 '[8] Ensure the range constraint of 1 ≤ n ≤ 600']

In [45]:
#以下是gpt构造器_单个subgoal生成
import openai

import json

save_pth='new_dataset/'

class gpt_interface:
    def __init__(self):
        openai.api_base = "https://api2.aigcbest.top/v1"
        self.client = openai.Client()
        try:
            with open(save_pth+'history_2.json','r') as f:
                self.history2=json.load(f)
        except:
            self.history2={}
        try:
            with open(save_pth+'history_3.json','r') as f:
                self.history3=json.load(f)

        except:
            
            self.history3 = {}
    def get_response(self,question,id,replay=False):
        
        messages=[
            {
                "role": "system",
                "content": '''
You are a math assistant. Please give me all the subgoals of the proof based on the question below. 
You can analysis the probelm as you want, but you must give this kind of output in the end. 
You should only output the equations of the subgoal, don't use too many text
If the subgoal is too simple, you can skip it.
Here is an example:
input : 
theorem aime_problem (hx : x > 1) (hy : y > 1) 
  (h1 : (log (y ^ x)) / (log x) = 10)
  (h2 : (log (x ^ (4 * y))) / (log y) = 10) :
  x * y = 25 := by

output:
Subgoals:
```subgoals
[1] log x > 0
[2] log y > 0
[3] (x * log y) / (log x) = 10
[4] (4 * y * log x) / (log y) = 10
[5] ((x * log y) / (log x)) * ((4 * y * log x) / (log y)) = 100
[6] 4 * x * y = 100
```
Now, please give me all the subgoals of this problem:
'''
            },
            {
                "role": "user",
                "content": question
            }
        ]
        if question in self.history2 and not replay:
            messages.append(
             {   "role": "system",
                "content": self.history2[question]})
            return messages,id
        #不在，或者没开启强制重放，则调用gpt

        response = self.client.chat.completions.create(
        model=["claude-3-5-haiku-20241022","gpt-4o",'claude-3-haiku-20240307'][0],
        temperature=0,
        messages=messages
    )
        result_str=(response.choices[0].message.content)
        self.history2[question]=result_str
        messages.append(
            {   "role": "system",
                "content": result_str})
        self.save()
        return messages,id
    def get_details(self,message,indexs,id):
        key = message[-1]['content']+str(indexs)
        if key in self.history3:
            return self.history3[key],id
        message.append({'role':'user','content':'''
Now you need to help me refine the proof for the following subgoals.
You do not need to give lean code. Just give me the refined proof for the subgoal.                        
 Here is an example:
input: refine [6] 4 * x * y = 100 
output:
```                       
[1] (4 * x * y) * ((log y * log x) / (log x * log y)) = 100
[2] (log y * log x) / (log x * log y) = 1
[3] (4 * x * y) * (1) = 100
```
Now, please help me refine the proof for the following subgoals: %d.                            
                        '''%indexs})
        response = self.client.chat.completions.create(
        model=["claude-3-5-haiku-20241022","gpt-4o",'claude-3-haiku-20240307'][0],
        messages=message
    )
        result_str=(response.choices[0].message.content)
        
        self.history3[key]=result_str
        self.save()
        return result_str,id


    def save(self):
      with open(save_pth+'history_2.json','w') as f:
          json.dump(self.history2,f)
      with open(save_pth+'history_3.json','w') as f:
          json.dump(self.history3,f)

In [46]:
#以下是多线程构造器
import concurrent.futures
def parallel_requests(func,prompts_list,max_process=16):
    results = [0]*len(prompts_list)
    # 使用线程池来并行处理请求
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_process) as executor:
        # 提交所有的请求到线程池
        futures = [executor.submit(func,item[0],item[1],id) for id,item in enumerate(prompts_list)]
        # 收集所有请求的结果
        for future in concurrent.futures.as_completed(futures):
            try:
                result,id = future.result()
                results[id]= result
            except Exception as exc:
                pass

    return results

In [47]:
test1000 = gpt_interface()

In [48]:
with open('/home/limengze/chen_dongheng/pipeline/new_dataset/MATH_TEST.jsonl', 'r') as f:
    data2 = [json.loads(l) for l in f.readlines()]
with open('/home/limengze/chen_dongheng/pipeline/final_data/MATH_TEST_LATEX_0.jsonl', 'r') as f:
    data = [json.loads(l) for l in f.readlines()]
# 构造数据对
#构造题目提问序列
prompts=[]
for item in data2:
    a=item['formal_statement']
    #去掉前两行
    # if item['category']=='aime_amc':
    #     a=a.split('\n')[2:]
    #     a='\n'.join(a)
    a=a.replace('import Mathlib\nopen BigOperators Real Nat Topology Rat\n','')
    prompts.append(a)


In [10]:
#构造请求序列
refine = []
for i in range(len(prompts)):
    subgoal = data[i]['latex']
    req=seg_text(subgoal)
    for j in range(len(req)):
        message = test1000.get_response(prompts[i],0)[0]
        refine.append((message,j+1))


In [49]:
len(refine)

3267

In [30]:
with open('/home/limengze/chen_dongheng/pipeline/final_data/refine_test_src.json','w') as f:
    json.dump(refine,f)

In [50]:
results = parallel_requests(test1000.get_details,refine,32)
len(test1000.history3)

3291

In [52]:
import re
from tqdm import tqdm
for i in tqdm(range(len(results))):
    if re.search(r'```(.*?)\n(.*?)\n```', results[i], re.DOTALL) == None:
        # print(re.search(r'```(.*?)\n(.*?)\n```', results[i], re.DOTALL).group(2))
        

100%|██████████| 3267/3267 [00:00<00:00, 69223.85it/s]

651
714
1057
1727
1764
1766
2053
2537
3059
3060
3264


# 构造half_shot
分成前半部分和后半部分，然后进行构造。

In [54]:
def con2(a,b):
    results=[]
    for i in range(1,len(a)):
        for j in range(i):
            lena = int(j*len(a)/len(b))
            lenb = int(i*len(b)/len(a))
            con = a[:j]+[b[lena]]+a[j+1:i]+[b[lenb]]+a[i+1:]
            hint=''.join(con)
            results.append({'hint':hint,'id0':i,'id1':j})
    return results

In [53]:
# 构造1-n的幂集
def powerset(n,maxn = 2):
    res = []
    for i in range(1<<n):
        cur = []
        for j in range(n):
            if i & (1<<j):
                cur.append(j)
        if len(cur)<=maxn:
            res.append(cur)
    return res

[[], [0], [1], [0, 1], [2], [0, 2], [1, 2]]

In [55]:
new_con2=[]
id = 0
for i,item in enumerate(data):
    a=item['latex']
    plist = seg_text(a)
    refine_set = powerset(len(plist),2)
    #裁切结果
    refine_results = results[id:id+len(refine_set)]
    # 逐个构造
    for ss in refine_set:
        plist2 = plist.copy()
        for j in ss:
            plist2[j] = re.search(r'```(.*?)\n(.*?)\n```', refine_results[j], re.DOTALL).group(2)
        # 拼合plist
        slgItem = item.copy()
        new_name = item['name']+'#'+str(ss)
        slgItem['name'] = new_name
        slgItem['formal_statement'] = data2[i]['formal_statement']
        # 融合提示词
        subsgoals = slgItem['formal_statement']+'\n'
        lines = ('\n'.join(plist2)).split('\n')
        for line in plist2:
            temp = line[line.find(']')+1:]
            subsgoals = subsgoals + '--' + temp + '\n'
        slgItem['formal_statement'] = subsgoals
        new_con2.append(slgItem)

    id+=len(plist)
    

AttributeError: 'NoneType' object has no attribute 'group'

In [69]:
# 将新的数据写入文件
with open('final_data/MATH_TEST_REFINE_0.jsonl','w') as f:
    for item in new_con2:
        f.write(json.dumps(item)+'\n')

In [87]:
print(new_con2[3]['formal_statement'])

import Mathlib
open BigOperators Real Nat Topology Rat
theorem remainder_problem (a : ℕ → ℕ) 
(h_def : ∀ n, a n = 6^n + 8^n) 
(h_mod : a 83 ≡ 35 [MOD 49]) : 
a 83 % 49 = 35 := by 
-- a 83 = 6^83 + 8^83          (by definition of function a)
[2] Expand 6^83 = 6 * 6^82      (using exponent rule)
[3] Expand 8^83 = 8 * 8^82      (using exponent rule)
[4] Recursively break down 6^82 and 8^82 
[5] Show that the expansion follows the pattern a(n) = 6^n + 8^n
[6] Use computational verification or mathematical induction to confirm the equality
-- Expand the definition of modular arithmetic
[2] Use properties of exponentiation modulo a number
[3] Break down 6^83 and 8^83 into their modular representations
[4] Apply Euler's theorem or fermat's little theorem for modular exponentiation
[5] Reduce the problem to finding (6^83 mod 49) and (8^83 mod 49)
[6] Use computational techniques or properties of 49 (7^2) to simplify the calculation
[7] Show that (6^83 mod 49) + (8^83 mod 49) = 35
-- (6^83 % 49

In [54]:
i

359

In [32]:
# with open('1127con/requests.json','w') as f:
#     json.dump(results,f)
with open('1127con/requests.json','r') as f:
    results=json.load(f)

seg_map = {}
for i in range(len(results)):
    name = datas[i]['name']
    try:
        seg_map[name] = seg_text(results[i])
    except:
        seg_map[name] = results[i]

In [34]:
with open('1127con/seg_map.json','w') as f:
    json.dump(seg_map,f)

# 回写第一个二分分割，首先需要加上一个自动注释器

In [14]:
def auto_comment(text):
    lines = text.splitlines()
    new_lines=[]
    for line in lines:
        if line.startswith('--'):
            new_lines.append(line)
        elif line.strip()=='':
            continue
        else:
            new_lines.append('-- '+line)
    return '\n'.join(new_lines)

In [15]:
new_ds_1=[]
for i in range(len(datas)):
    new_obj=datas[i].copy()
    new_obj2=datas[i].copy()
    new_obj['formal_statement']+='\n%s\n'%auto_comment(new_con1[i][0])
    new_obj2['formal_statement']+='\n%s\n'%auto_comment(new_con1[i][1])
    new_obj['kname']=new_obj['name']+'F'
    new_obj2['kname']=new_obj['name']+'B'
    new_ds_1.append(new_obj)
    new_ds_1.append(new_obj2)

In [28]:
new_ds_2=[]
for i in range(len(datas)):
    if type(new_con2[i])==str:
        new_obj=datas[i].copy()
        new_obj['formal_statement']+='\n%s\n'%auto_comment(new_con2[i])
        new_obj['kname']=new_obj['name']
        new_ds_2.append(new_obj)
        continue
    for j in range(len(new_con2[i])):
        new_obj=datas[i].copy()
        new_obj['formal_statement']+='\n%s\n'%auto_comment(new_con2[i][j]['hint'])
        new_obj['kname']=new_obj['name']+'_%d_%d'%(new_con2[i][j]['id1'],new_con2[i][j]['id0'])
        new_obj['src_hints']=seg_text(results[i])
        new_ds_2.append(new_obj)

In [17]:
new_obj

{'name': '2000_AIME_I_10',
 'split': 'test',
 'informal_prefix': 'A sequence of numbers $x_{1},x_{2},x_{3},\\ldots,x_{100}$ has the property that, for every integer $k$ between $1$ and $100,$ inclusive, the number $x_{k}$ is $k$ less than the sum of the other $99$ numbers. Given that $x_{50} = \\frac{m}{n}$, where $m$ and $n$ are relatively prime positive integers, find $m + n$.',
 'formal_statement': 'import Mathlib\nopen BigOperators Real Nat Topology Rat\ntheorem aime_problem_934 (m n : ℕ) (x : Fin 100 → ℚ)\n  (hmn_pos : m > 0 ∧ n > 0)\n  (h_coprime : Nat.Coprime m n)\n  (h_seq : ∀ k : Fin 100, \n    x k = (∑ i : Fin 100, x i) - x k - k)\n  (h_x50 : x ⟨49, ⟩ = m / n) :\n  m + n = 173 := by\n  \n-- 1. **Prove the Expression \\(\\sum_{i=1}^{100} x_i = 2x_k + k\\) for All \\( k \\):**\n--    The problem states: for each \\( k \\), \n--    \\[\n--    x_k = \\sum_{i=1, i \\neq k}^{100} x_i - k\n--    \\]\n--    which can be rewritten as:\n--    \\[\n--    \\sum_{i=1, i \\neq k}^{100} x_i

In [18]:
seg_text(results[i])

['\n1. **Prove the Expression \\(\\sum_{i=1}^{100} x_i = 2x_k + k\\) for All \\( k \\):**\n\n   The problem states: for each \\( k \\), \n\n   \\[\n   x_k = \\sum_{i=1, i \\neq k}^{100} x_i - k\n   \\]\n\n   which can be rewritten as:\n\n   \\[\n   \\sum_{i=1, i \\neq k}^{100} x_i = x_k + k\n   \\]\n\n   Thus, the total sum of all the numbers is:\n\n   \\[\n   \\sum_{i=1}^{100} x_i = x_k + \\sum_{i=1, i \\neq k}^{100} x_i = x_k + (x_k + k) = 2x_k + k\n   \\]\n\n   Hence, this equation holds for all \\( k \\).\n',
 '\n2. **Use \\( x_{50} = \\frac{m}{n} \\) to Find an Expression for \\(\\sum_{i=1}^{100} x_i\\):**\n\n   By setting \\( k = 50 \\) in the formula from step 1:\n\n   \\[\n   \\sum_{i=1}^{100} x_i = 2x_{50} + 50 = 2\\left(\\frac{m}{n}\\right) + 50 = \\frac{2m}{n} + 50\n   \\]\n\n   Therefore, the entire sum: \n\n   \\[\n   S = \\frac{2m}{n} + 50\n   \\]\n',
 '\n3. **Establish a Relation Between \\(\\sum_{i=1}^{100} x_i\\) and \\(m, n\\):**\n\n   Similarly, apply the formula for

In [19]:
#根据skip_list 进行过滤
with open('1127con/skip_list.json','r') as f:
    skip_list=json.load(f)

In [20]:
new1=[]
for item in new_ds_1:
    if item['name'] not in skip_list:
        temp = item.copy()
        temp ['name'] = temp['kname']
        new1.append(temp)

In [29]:
new2=[]
for item in new_ds_2:
    if item['name'] not in skip_list:
        temp = item.copy()
        temp ['name'] = temp['kname']
        new2.append(temp)

In [22]:
len(new2)

6868

In [30]:
#保存数据
with open('1127con/aime_amc_new_con1.jsonl','w') as f:
    for item in new1:
        f.write(json.dumps(item)+'\n')
with open('1127con/aime_amc_new_con2.jsonl','w') as f:
    for item in new2:
        f.write(json.dumps(item)+'\n')